<h1> DattoRMM - MS Patching Timeseries </h1>

## Import Modules

In [ ]:
import datetime
import re
import os
import csv
import sklearn as sk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.offline as offline
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import xlrd

## Arguments and Declarations


In [ ]:
# Selected Columns from Master Device View Export CSV - This is the column mask that will be used to trimp the outer merge on match key column
std_columns = [
    'Device UID',
    'Site Name',
    'Site UID',
    'Device Hostname',
    'Create Date',
    'Last Seen',
    'Last Audit Date',
    'Policy',
    'Patches Approved Pending',
    'Patches Not Approved',
    'Patches Installed',
    'Patch Status',
    'Schedule',
    'Last Run',
    'Operating System',
    'Device CPU',
    'Physical CPU Cores',
    '.NET Version',
    'Memory',
    'Device Type',
    'Domain',
    'Disk Drive (total/free)',
    'Online Duration (hrs)',
    'Architecture',
    'Last Reboot',
    'Reboot required',
    'Int IP Address',
    'User-Defined Field 10',
    'MAC Address(es)',
    'Software Status',
    'Group',
    'Antivirus Product',
    'Antivirus Status',
    'Source Modified Date',
    'Source Creation Date',
    'Source Filename'
 ]

In [ ]:
# review column filter for any duplicates values or close similarities
std_col_ser = pd.Series(std_columns).value_counts()
std_col_ser[std_col_ser > 1]

In [ ]:
# filename prefix timestamp format
time_format = '%Y_%m_%d_%H%M%S'

# define key column to join on
fieldnames_to_compare = 'Device UID'

# identify folder stages so that files are not called twice in the same stage
source_dir = 'D:/cloud_storage/Think Stack/Infrastructure - Documents/Reports - Archiving and Distribution'

# Parse Date Data Options
date_parser = lambda c: pd.to_datetime(c, errors='coerce')
parse_dates =  ['Create Date', 'Last Seen','Last Reboot']

# NA Values Check
na_values = ['Currently Online','null', '(null)']

# output of csv with matching key column
included_files = {}

# output csv of all files that could not be merged
excluded_files = {}

# Regex Match to group files to be combined on rows rather than merged on columns to prevent dropping rows if there isnt a key column match when files are combined in random order
pattern = re.compile(r'^(\w+)_')

# final dataframe before training and visualization
df_clean = pd.DataFrame(columns=std_columns)

<h1>Read all files in source_dir and sub directories</h1>
    <h3> Filter by '.csv' </h3>

In [ ]:
# pull all filenames walking through all folders (recursive going down the tree)
#all_source_csv = []
source_report_dict = {}
for root, dirs, files in os.walk(source_dir):
    for file in files:
        if ('.csv' in file) | ('.xlsx' in file):
            #all_source_csv.append(os.path.join(root,file))
            source_report_dict.update({os.path.join(file):os.path.join(root,file)})

# print(all_source_csv)
print('All CSV Files found before futher vetting and filtering')
print('='*50)
for file in source_report_dict:
    print(file)

<h1>Sorting and Excluding Files</h1>

## Read all csv file columns and create two lists of files:
### Those with the chosen merge key column will be kept and the remaining filenames will not be called any further

## Parse Accepted Reports for file discription and store as dictionary key pair

In [ ]:
string = 'ExampleFCU - 2020 - 11 - Patching Audit.xlsx'
ms_patch_prog = re.compile(r'(\bSIG\b|\bExampleFCU\b|\bExample FCU\b){1}.*(?!\b3rd\b)+(\bMS\b|\bPatch\b|\bPatching\b)+')
third_prog = re.compile(r'(\b3rd\b|\b3PP\b)+')
extension_prog = re.compile(r'(\.\w{3,4})')
report_date_prog = re.compile(r'(\d{4}).*(\d{2})')

In [ ]:
ms_patch_reports = []
for k,v in source_report_dict.items():
    match = {}

    if (ms_patch_prog.search(k) != None) & (third_prog.search(k) == None):

        # date nested dict
        date = {}
        date['year'] = str(report_date_prog.findall(k)[0][0])
        date['month'] = str(report_date_prog.findall(k)[0][1])

        match['filename'] = k
        match['path'] = v

        # extension
        if extension_prog.findall(file['filename']) == ['.xlsx']:
            match['extension'] = 'xlsx'

        elif extension_prog.findall(file['filename']) == ['.csv']:
            match['extension'] = 'csv'

        match['date'] = date

        ms_patch_reports.append(match)

## For those files that have the key column, set index col and add source info
### 1. Add source file data as columns at end of dataframe (record the file creation, modified, and fullpath name)
### 2. Set index col = fieldnames_to_compare variable list

In [ ]:
import os
import datetime

def AddTime(source_file):
    source_mfdate = 'Source Modified Date'
    source_crdate = 'Source Creation Date'
    source_fn = 'Source Filename'

    # Both the variables would contain time
    # elapsed since EPOCH in float
    ti_c = os.path.getctime(source_file)
    ti_m = os.path.getmtime(source_file)
    fi_n = os.path.basename(source_file)


    # Converting the time in seconds to UTC datetime
    c_ti = datetime.datetime.utcfromtimestamp(ti_c).strftime('%Y/%m/%d %H:%M:%S')
    m_ti = datetime.datetime.utcfromtimestamp(ti_m).strftime('%Y/%m/%d %H:%M:%S')


    return {source_crdate:c_ti,source_mfdate:m_ti,source_fn:fi_n}

In [ ]:
def map_source(source_file):
    # pull source time from file properties
    source_info = AddTime(source_file)

    # Import CSV
    df = pd.read_csv(source_file,index_col=fieldnames_to_compare)

    # add source info to new columns k with values v
    for k,v in source_info.items():
        df[k] = v

    return df

## Attempt 2 at updating data correctly on import.  Attempting to use iloc for each df row

In [ ]:
ms_patch_reports

In [ ]:
for df in ms_patch_reports:

    print(df['filename'])
    print(df['path'])
    print(df['date'])
    print(df['extension'])

In [ ]:
df2 = df.dropna(how='all',axis=1)

In [ ]:
# Create blank DataFrame to fill
df_master = pd.DataFrame(columns=std_columns)

In [ ]:
test_details = pd.read_csv('.csv')

In [ ]:
test_manage = pd.read_csv('.csv')

In [ ]:
for row in df_test1.row:
    print(row)

In [ ]:
for file in devices_tab_export_files:
    #print(file['filename'])
    #print(file['groupname'])
    df_devices_list.append(map_source(file['filename']))


## Attempt 1 at updating data correctly on import.  Still having issues with micromanaging file order on join

In [ ]:
#print(manage_tab_export_files)
#print(devices_tab_export_files)
for file in devices_tab_export_files:
    #print(file['filename'])
    #print(file['groupname'])
    df_devices_list.append(map_source(file['filename']))




for file in manage_tab_export_files:
    #print(file['filename'])
    #print(file['groupname'])
    df_manage_list.append(map_source(file['filename']))


#print(df_manage_list)

In [ ]:
df_devices = pd.concat(df_devices_list,axis=0)
df_manage = pd.concat(df_manage_list,axis=0)

In [ ]:
df_manage.info()

In [ ]:
df_manage

In [ ]:
df_devices.info()

<h1>Join, Concat, and Merge</h1>

## Those of the same name should be concatonated by row or stacked

## Merge all dataframes (csv's) into an empty dataframe that contains all columns without data

In [ ]:
# as a df (dataframe) must be merged on another, we start with filelist with the first element as the df all will be merged into index[0] in list dtype
df_clean = df_list[0]

for df_object in df_list[1:]:
    # join on key column or columns (original 'set' dtype must be changed to 'list' to fit pandas expected argument for 'merge' method)
    df_clean.merge(df_object, on=(fieldnames_to_compare), how= 'outer',suffixes=('', '_drop'))

In [ ]:
# drop duplicate colummns renamed as '_drop' during parse
df_clean.drop([col for col in df_clean.columns if 'drop' in col], axis=1, inplace=True)

## Trim any columns not in the column standars list 'columns'

In [ ]:
len(df_clean.columns)

In [ ]:
df_clean.drop([col for col in df_clean.columns if col not in std_columns], axis=1, inplace=True)

In [ ]:
len(df_clean.columns)

In [ ]:
# review column filter for any duplicates columns and drop one
clean_col = dict(df_clean.columns.value_counts())
dup_cols = list({k for (k,v) in clean_col.items() if v > 1})
if dup_cols:
    df_clean.rename(dup_cols,axis=1,inplace=True)
df_clean

## Using lists 'parse_dates' as datetime column targets and 'data_parser' as the datetime function to be applied to each value in target columns along each row that will change value type

In [ ]:
# convert dates to datetime
# variable 'time_format' stated at declaration
df_clean[parse_dates] = df_clean[parse_dates].apply(date_parser)

## Add 'Offline 30 days' and 'Extended Reboot' Columns as datetime delta calculations from day this report is run

In [ ]:
# Fill in NaT / NaN data with 0 time so columns can be converted to datetime and datetime methods can be used
df_clean[parse_dates].fillna(pd.Timedelta('0 days'),inplace=True)

# Filter - Devices Offline 30 Days
df_clean['Offline 30 Days'] = df_clean['Last Seen'] > datetime.datetime.now() - pd.to_timedelta("30day")

# Filters - Last Reboot Extended Duration and Online without Reboot Extended Duration
df_clean['Last Reboot Extended'] = df_clean['Last Reboot'] > datetime.datetime.now() - pd.to_timedelta("30day")

## Apply heatmap to review any NaN or NaT (null) values before they can be dropped

In [ ]:
plt.figure(figsize=(20,10))
sns.heatmap(data = df_clean.isnull(),yticklabels=False,cbar=False,cmap='plasma')

<h1>Start ML Trials</h1>

In [ ]:
## Create MS Patching pairplot DataFrame
df_patch_pair = df_clean

## Convert 'Category' columns into numbers to get value relationships

In [ ]:
d_patch_status = pd.get_dummies(df_clean['Patch Status'],prefix='d',prefix_sep='_')
df_patch_pair.drop('Patch Status',axis=1,inplace=True)
df_patch_pair = pd.concat([df_patch_pair,d_patch_status])

In [ ]:
d_offline_30days = pd.get_dummies(df_clean['Offline 30 Days'],drop_first=True,prefix='d',prefix_sep='_')
d_offline_30days.rename(columns={'d_True':'d_Offline_30 Days'},inplace=True)
df_patch_pair.drop('Offline 30 Days',axis=1,inplace=True)
df_patch_pair = pd.concat([df_patch_pair,d_offline_30days])

In [ ]:
d_last_reboot_ext = pd.get_dummies(df_clean['Last Reboot Extended'],drop_first=True,prefix='d',prefix_sep='_')
d_last_reboot_ext.rename(columns={'d_True':'d_Last Reboot Extended'},inplace=True)
df_patch_pair.drop('Last Reboot Extended',axis=1,inplace=True)
df_patch_pair = pd.concat([df_patch_pair,d_last_reboot_ext])

In [ ]:
d_av_status = pd.get_dummies(df_clean['Antivirus Status'],drop_first=True,prefix='d',prefix_sep='_')
d_av_status.rename(columns={'d_Running & up-to-date':'d_AV Status Ok'},inplace=True)
df_patch_pair.drop('Antivirus Status',axis=1,inplace=True)
df_patch_pair = pd.concat([df_patch_pair,d_av_status])

In [ ]:
d_reboot_required = pd.get_dummies(df_clean['Reboot required'],drop_first=True,prefix='d',prefix_sep='_')
d_reboot_required.rename(columns={'d_True':'d_Reboot required'},inplace=True)
df_patch_pair.drop('Reboot required',axis=1,inplace=True)
df_patch_pair = pd.concat([df_patch_pair,d_reboot_required])
d_reboot_required

In [ ]:
sns.pairplot(data=df_patch_pair)

In [ ]:
for col in df_patch_pair.columns:
    print(col)

In [ ]:
sns.set_style('darkgrid')
plt.figure(figsize=(30,15))
sns.lineplot(data=df_patch_pair,x='Last Reboot',y='Patches Approved Pending',lw=.5)
plt.savefig('.png')

In [ ]:
df_patch_pair['Online Duration (hrs)'].value_counts()

In [ ]:
df_patch_pair['Site Name'].unique()

In [ ]:
sns.set_style('darkgrid')
plt.figure(figsize=(30,15))
sns.barplot(data=df_patch_pair,x='Site Name',y='Online Duration (hrs)')

In [ ]:
# store filename info as dictionary
var_dict = {}
i = 0
df_list = []
df_names =[]


# create variable dictionary before import iteration
for filename in all_shaped_csv:
    var_dict.update({('df' + str(i)): (shaped_dir + filename)})
    i = i + 1

# for filename = key(k) import into pandas and append resulting dataframe to df list as element
for k, v in var_dict.items():
    k = pd.read_csv(v)
    df_names.append(v)
    df_list.append(k)

#print(df_names)

# as a df (dataframe) must be merged on another, we start with filelist with the first element as the df all will be merged into index[0] in list dtype
for df_object in df_list[1:]:
    #print(df_object)
    #print("")

    # join on key column or columns (original 'set' dtype must be changed to 'list' to fit pandas expected argument for 'merge' method)
    df_list[0].merge(df_object, on=list(fieldnames_to_compare), how= 'outer')
    print(df_list[0]['Policy'])


# add current timestamp to filename for reference
current_time = (datetime.datetime.utcnow().strftime('%Y_%m_%d_%H%M%S'))

# add 'merged_' to filename startswith and export
df_list[0].to_csv(merged_dir + 'merged_' + str(current_time) + ".csv", index= False)

cleanup = True
# clean up intermediate data
if cleanup == True:
    for s in all_shaped_csv:
        path = shaped_dir + s
        print(f'Removing file {path}')
        os.remove(path)







In [ ]:
df

In [ ]:
for filename in all_source_csv:
    source_info = srctime(source_dir + filename)
    df = pd.read_csv(source_dir + filename)
    for k,v in source_info.items():
        print(f'The key is {k}')
        print(f'The value is {v}')
        df[k] = v
    #df.insert(1,columns=source_info.keys(),source_info.values())
    #df[source_info.keys()]
    #print(source_info)
    #print(source_info.keys())
    #print(source_info.values())


    #print(filename)

    #df.colums = fs.patch_columns
    #print(df.columns)

In [ ]:
df